Developing "Momentum" Trading Strategy
 - Many differnt takes on this strategy
 - Sometimes referred to as trend following
 - Whatever you do and call it, you first need to research potential trading signals

In [5]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt

1. Download data:
Many services for this, some paid some free
Yahoo Finance API
Typically trading "systems" involve a number of securities
For this demonstration we are just going to look at GLD --> the gold ETF

In [ ]:
gld = pdr.get_data_yahoo('GLD')
day = np.arange(1, len(gld) + 1)
gld['day'] = day
gld.drop(columns=['Adj Close', 'Volume'], inplace = True)
gld = gld[['day', 'Open', 'High', 'Low', 'Close']]
gld.head()

In [ ]:
gld.info()

2. Add data/transform data
calculate signal based on some price or statistical action
we are going to try a moving average crossover to generate signals
for this strategy we will always by "in" a trade, either long or short
we are modeling; this means real life variation should be expected
Add moving averages to the data frame

In [ ]:
gld['9-day'] = gld['Close'].rolling(9).mean()
gld['21-day'] = gld['Close'].rolling(21).mean()
gld[19:25]

In [ ]:
#Add "signal" column
gld['signal'] = np.where(gld['9-day'] > gld['21-day'], 1, 0)
gld['signal'] = np.where(gld['9-day'] < gld['21-day'], -1, gld['signal'])
gld.dropna(inplace=True)
gld.head()

In [ ]:
#Calculate Instantaneous returns/system returns
gld['return'] = np.log(gld['Close']).diff()
gld['system_return'] = gld['signal'] * gld['return']
gld['entry'] = gld.signal.diff()
gld.head()

In [ ]:
#Plot trades on time series
plt.rcParams['figure.figsize'] = 12, 6
plt.grid(True, alpha = .3)
plt.plot(gld.iloc[-252:]['Close'], label = 'GLD')
plt.plot(gld.iloc[-252:]['9-day'], label = '9-day')
plt.plot(gld.iloc[-252:]['21-day'], label = '21-day')
plt.plot(gld[-252:].loc[gld.entry == 2].index, gld[-252:]['9-day'][gld.entry == 2], '^',
         color = 'g', markersize = 12)
plt.plot(gld[-252:].loc[gld.entry == -2].index, gld[-252:]['21-day'][gld.entry == -2], 'v',
         color = 'r', markersize = 12)
plt.legend(loc=2);

In [ ]:
plt.plot(np.exp(gld['return']).cumprod(), label='Buy/Hold')
plt.plot(np.exp(gld['system_return']).cumprod(), label='System')
plt.legend(loc=2)
plt.grid(True, alpha=.3)

In [ ]:
np.exp(gld['return']).cumprod()[-1] -1

In [ ]:
np.exp(gld['system_return']).cumprod()[-1] -1